In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
data_dir = "srchive (2)/data"
img_size = (128, 128)
batch_size = 32
num_epochs = 20
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",
    color_mode="rgb"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",
    color_mode="rgb"
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Class names:", class_names)
data_augmentation = Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

normalization_layer = layers.Rescaling(1./255)

def preprocess_train(x, y):
    x = normalization_layer(x)
    x = data_augmentation(x)
    return x, y

def preprocess_val(x, y):
    x = normalization_layer(x)
    return x, y

train_ds = train_ds.map(preprocess_train).shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess_val).prefetch(tf.data.AUTOTUNE)
model = Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=img_size + (3,)),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "best_static_cnn_colab.h5",
    monitor='val_loss',
    save_best_only=True
)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=num_epochs,
    callbacks=[early_stop, checkpoint]
)
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {acc*100:.2f}%")

model.save("static_cnn_colab_final.h5")


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/mahmoudreda55/satellite-image-classification
License(s): copyright-authors
  0% 0.00/21.8M [00:00<?, ?B/s]
100% 21.8M/21.8M [00:00<00:00, 646MB/s]
/content/satellite_data
['data']
Found 5631 files belonging to 4 classes.
Using 4505 files for training.
Found 5631 files belonging to 4 classes.
Using 1126 files for validation.
Class names: ['cloudy', 'desert', 'green_area', 'water']


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,517,060 (24.86 MB)

...

Epoch 1/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 961ms/step - accuracy: 0.6585 - loss: 0.8098

141/141 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - accuracy: 0.6591 - loss: 0.8076 - val_accuracy: 0.8996 - val_loss: 0.3177
Epoch 2/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 943ms/step - accuracy: 0.8969 - loss: 0.2454

141/141 ━━━━━━━━━━━━━━━━━━━━ 170s 1s/step - accuracy: 0.8969 - loss: 0.2453 - val_accuracy: 0.8925 - val_loss: 0.2868
Epoch 3/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 960ms/step - accuracy: 0.9244 - loss: 0.1796

141/141 ━━━━━━━━━━━━━━━━━━━━ 203s 1s/step - accuracy: 0.9244 - loss: 0.1796 - val_accuracy: 0.9130 - val_loss: 0.2171
Epoch 4/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 166s 1s/step - accuracy: 0.9265 - loss: 0.1553 - val_accuracy: 0.7922 - val_loss: 0.6529
Epoch 5/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.9157 - loss: 0.1948 - val_accuracy: 0.8925 - val_loss: 0.2582
Epoch 6/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.9232 - loss: 0.1679 - val_accuracy: 0.8561 - val_loss: 0.2790
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 266ms/step - accuracy: 0.9065 - loss: 0.2249


Validation Accuracy: 91.30%


In [4]:
model.save("static_cnn_colab_final.keras")


In [5]:
files.download("static_cnn_colab_final.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>